# Time series - multivariate prediction, LSTM for continuous prediction

In this example we will load prepared data and try to predict it with a lstm network producing continuous prediction

This example is very similar to the last lstm example except the model is trained to predict one step into the future that can be used for latter predictions.

In [ ]:
import numpy as np
import os
from datetime import date
import re

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
pd.options.display.max_rows = 30

In [ ]:
training_df = pd.read_pickle("../data/weather/ts_temp_dp_press.p")

Notice that we would randomly sample the training data from the whole weather history.

The `indices` array will contain starting date for each sample

In [ ]:
import random

PREDICTION_LEN = 7
HISTORY_LEN = 14
NUM_SAMPLES = 2000
NUM_INPUTS = len(training_df.columns)

indices = random.sample(range(0,len(training_df) - HISTORY_LEN - 1), NUM_SAMPLES)

In this example the target sequence is shifted one step into the future. The model need to accumulate information in its inner state to correctly predict it.

Both input and output tensors remain in sequence form.

In [ ]:
ts_data = training_df.values
ts_inputs = #TODO: add correctly selected input data based on indices
ts_outputs = #TODO: add correctly selected output data based on indices (one step ahead forecact)

In [ ]:
#TODO: build model for one step ahead forecast. (many->many)

In [ ]:
model.fit(ts_inputs, ts_outputs, validation_split=0.1, epochs=100)

In [ ]:
model.save_weights('model_plus1_predition.hdf')

In [ ]:
#TODO: build model for one step ahead forecast but use stateful  (many->many)

pred_model = Model(inputs, outputs)
pred_model.summary()
pred_model.load_weights('model_plus1_predition.hdf')

## Prediction phase

To test the prediction, we will take a particular date and feed the history into the network.

The network will predict one step into the future and we will take this prediction and feed it back to the network. In this way, we can predict multiple (and variable) steps into the future

Notice that the model is `stateful` i.e. the hidden state does not reset after the prediction.

In [ ]:
import datetime
from datetime import timedelta

In [ ]:
prediction_date = date(2014, 12, 23)

prediction_input = training_df[prediction_date - timedelta(days=HISTORY_LEN - 1): prediction_date]

pred_model.reset_states()
for i in range(0, HISTORY_LEN):
    prediction = pred_model.predict(prediction_input.values[i].reshape(1, 1, NUM_INPUTS))
    
self_predicition = [np.copy(prediction)] #add to this list in the loop below

for i in range(0, PREDICTION_LEN - 1):
    #TODO: forecast into the future by feeding the model its own output
    
history = training_df[prediction_date - timedelta(days=HISTORY_LEN - 1): prediction_date + timedelta(days=PREDICTION_LEN)]
prediction_df = pd.DataFrame(np.vstack(self_predicition).reshape((PREDICTION_LEN, NUM_INPUTS)), index=pd.DatetimeIndex(start=prediction_date + timedelta(days=1), freq='D', periods=PREDICTION_LEN))


In [ ]:
plt.figure(figsize = (12, 6))
history_plt = plt.plot(history.iloc[:,0:3], linestyle='--')
plt.plot(history.iloc[:,3], 'oc')
plt.plot(prediction_df.iloc[:,0], color=history_plt[0].get_color())
plt.plot(prediction_df.iloc[:,1], color=history_plt[1].get_color())
plt.plot(prediction_df.iloc[:,2], color=history_plt[2].get_color())
plt.plot(prediction_df.iloc[:,3], 'or')
plt.show()